In [1]:
import pymongo
from superduperdb import superduper


In [2]:
# setup SuperDuperDB

db = pymongo.MongoClient().documents
db = superduper(db)



NotImplementedError: Couldn't auto-identify Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'documents'), please wrap explicitly using ``superduperdb.core.*``

In [3]:
from sklearn.datasets import load_iris
from superduperdb.core.document import Document
from superduperdb.datalayer.mongodb.query import Collection

X, y = load_iris(return_X_y=True)



In [4]:
from sklearn.datasets import load_iris
from superduperdb.core.document import Document
from superduperdb.datalayer.mongodb.query import Collection
from superduperdb.encoders.numpy.array import array


# Load the [iris dataset](https://scikit-learn.org/stable/datasets/toy_dataset.html#iris-plants-dataset)
X, y = load_iris(return_X_y=True)
encoder = array(dtype='float64', shape=(4,))

# Create a MongoDB document for each row in the dataset
collection = Collection(name='iris')
data = [Document({'x': encoder(X_value), 'y':int(y_value) }) for (X_value, y_value) in zip(X,y)]

db.execute(
    collection.insert_many(data, encoders=[encoder])
)


INFO:root:found 0 uris


(<pymongo.results.InsertManyResult at 0x2b1f898d0>,
 TaskWorkflow(database=<superduperdb.datalayer.base.datalayer.Datalayer object at 0x1687cf510>, G=<networkx.classes.digraph.DiGraph object at 0x168808250>))

In [5]:
r = db.execute(collection.find_one())
r

Document({'_id': ObjectId('64bea463e6c6317e8fddb959'), 'x': Encodable(x=array([5.1, 3.5, 1.4, 0.2]), encoder=Encoder(identifier='numpy.float64[4]', decoder=<Artifact artifact=<superduperdb.encoders.numpy.array.DecodeArray object at 0x2b1fbb7d0> serializer=dill>, encoder=<Artifact artifact=<superduperdb.encoders.numpy.array.EncodeArray object at 0x2b1fbac50> serializer=dill>, shape=[4], version=5)), 'y': 0, '_fold': 'train'})

In [6]:
from sklearn.linear_model import LogisticRegression

superduper_clf = superduper(LogisticRegression())


superduper_clf.fit(
    X='x',
    y='y',
    db=db,
    select=Collection(name='iris').find(),
)



ServerSelectionTimeoutError: localhost:27018: [Errno 61] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 64beae7a061c4236c9dbedcf, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27018) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27018: [Errno 61] Connection refused')>]>

In [8]:
from sklearn.linear_model import LogisticRegression
from superduperdb.models.sklearn.wrapper import Estimator

superduper_clf = Estimator(identifier='LR', object=LogisticRegression())


superduper_clf.fit(
    X='x',
    y='y',
    db=db,
    select=Collection(name='iris').find(),
)



ServerSelectionTimeoutError: localhost:27018: [Errno 61] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 64beade7c471bb957f768a1a, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27018) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27018: [Errno 61] Connection refused')>]>

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# create a pipeline object
pipe = make_pipeline(StandardScaler(),
     LogisticRegression()
 )
# load the iris dataset and split it into train and test sets
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
# fit the whole pipeline
pipe.fit(X_train, y_train)
# we can now use it like any other estimator
accuracy_score(pipe.predict(X_test), y_test)

0.9736842105263158